In [1]:
import uproot
import numpy as np
import torch

from tensorflow.keras.models import load_model

from HigherTier import HigherTierFileHelper
import GraphBuilder
import Models
import MPNN

In [2]:
#######################################
# Create our models
#######################################

############################
# Config 
############################
primaryFileSuffix = '_ParticleToNeutrinoTruth.pt'

# For primary GNN
NUM_LAYERS_GNN = 4
EMB_DIM_GNN = 16
INPUT_DIM_GNN = 23 # node features
EDGE_DIM_GNN = 1
GNN_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/gnn_" + primaryFileSuffix

# For primary edge classification model
PRIMARY_EDGE_CLASSIFIER_MODEL_PATH = "/Users/isobel/Desktop/DUNE/2024/Hierarchy/models/edge_classifier_" + primaryFileSuffix
OTHER_EDGE_CLASSIFIER_MODEL_PATH = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/models/other_simple_model'

PRIMARY_THRESHOLD = 0.4
OTHER_THRESHOLD = 0.8

############################
# Setup the models for eval
############################

gnn_model = MPNN.MPNNModel(num_layers=NUM_LAYERS_GNN, emb_dim=EMB_DIM_GNN, input_dim=INPUT_DIM_GNN, edge_dim=EDGE_DIM_GNN)
primary_edge_classifier_model = Models.EdgeClassifier()
other_edge_classifier_model = load_model(OTHER_EDGE_CLASSIFIER_MODEL_PATH)

gnn_model.load_state_dict(torch.load(GNN_MODEL_PATH))
gnn_model.eval()

primary_edge_classifier_model.load_state_dict(torch.load(PRIMARY_EDGE_CLASSIFIER_MODEL_PATH))
primary_edge_classifier_model.eval()

EdgeClassifier()

In [3]:
#############################
# Let's read the file
#############################

#fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/nu_dune10kt_1x2x6_1413_998_20230826T184801Z_gen_g4_detsim_hitreco__20240229T163039Z_reco2_ccnutree'

fileName = '/Users/isobel/Desktop/DUNE/2024/Hierarchy/HigherTier/files/without2DPFPs/ccnutree_0'

inputFileName = fileName + '.root'

treeFile = uproot.open(inputFileName)
tree = treeFile['ccnuselection/ccnusel']
branches = tree.arrays()

In [4]:
#############################
# Get event-level stuff
#############################
run = np.array(branches['Run'])
subrun = np.array(branches['SubRun'])
event = np.array(branches['Event'])

recoNuVertexX = branches['RecoNuVtxX']
recoNuVertexY = branches['RecoNuVtxY']
recoNuVertexZ = branches['RecoNuVtxZ']


recoGeneration_main = branches['RecoPFPRecoGeneration']
    
#############################
# Get pfp-level stuff - these cannot be numpy arrays...
#############################
trackShowerScore_main = branches['RecoPFPTrackShowerScore']
nHits_main = branches['RecoPFPRecoNHits']
charge_main = branches['RecoPFPRecoCharge']
vertexX_main = branches['RecoPFPRecoVertexX']
vertexY_main = branches['RecoPFPRecoVertexY']
vertexZ_main = branches['RecoPFPRecoVertexZ']
trackEndX_main = branches['RecoTrackRecoEndX']
trackEndY_main = branches['RecoTrackRecoEndY']
trackEndZ_main = branches['RecoTrackRecoEndZ']
showerDirX_main = branches['RecoShowerRecoDirX']  # not the best direction estimate, placeholder
showerDirY_main = branches['RecoShowerRecoDirY']
showerDirZ_main = branches['RecoShowerRecoDirZ']
ivysaurusMuon_main = branches['RecoPFPIvysaurusMuon']
ivysaurusProton_main = branches['RecoPFPIvysaurusProton']
ivysaurusPion_main = branches['RecoPFPIvysaurusPion']
ivysaurusElectron_main = branches['RecoPFPIvysaurusElectron']
ivysaurusPhoton_main = branches['RecoPFPIvysaurusPhoton']
wiggliness_main = branches['RecoTrackDeflecAngleSD']
trackLength_main = branches['RecoTrackLength']
displacement_main = branches['RecoShowerPandrizzleDisplacement']
dca_main = branches['RecoShowerPandrizzleDCA']
recoGeneration_main = branches['RecoPFPRecoGeneration']
recoSelf_main = branches['RecoPFPSelf']
pathwayLength_main = branches['RecoShowerPandrizzlePathwayLengthMin']
pathwayScatteringAngle2D_main = branches['RecoShowerPandrizzleMaxShowerStartPathwayScatteringAngle2D']
nShowerStartHits_main = branches['RecoShowerPandrizzleMaxNPostShowerStartHits']
showerScatterAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartScatterAngle']
nuVertexEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyAsymmetry']
showerStartEnergyAsymmetry_main = branches['RecoShowerPandrizzleMaxPostShowerStartShowerStartEnergyAsymmetry']
nuVertexEnergyWeightedMeanRadialDistance_main = branches['RecoShowerPandrizzleMaxPostShowerStartNuVertexEnergyWeightedMeanRadialDistance']
showerMoliereRadius_main = branches['RecoShowerPandrizzleMinPostShowerStartShowerStartMoliereRadius']
showerOpeningAngle_main = branches['RecoShowerPandrizzleMaxPostShowerStartOpeningAngle']
foundHitRatio_main = branches['RecoShowerPandrizzleMaxFoundHitRatio']
initialGapSize_main = branches['RecoShowerPandrizzleMaxInitialGapSize']
largestProjectedGapSize_main = branches['RecoShowerPandrizzleMinLargestProjectedGapSize']
nViewsWithAmbiguousHits_main = branches['RecoShowerPandrizzleNViewsWithAmbiguousHits']
ambiguousHitMaxUnaccountedEnergy_main = branches['RecoShowerPandrizzleAmbiguousHitMaxUnaccountedEnergy']

# True information.. (for cheating)
pfpTrueMomX_main = branches['RecoPFPTrueMomX']
pfpTrueMomY_main = branches['RecoPFPTrueMomY']
pfpTrueMomZ_main = branches['RecoPFPTrueMomZ']
pfpTruePDG_main = branches['RecoPFPTruePDG']

#############################
# Network truth - these cannot be numpy arrays...
#############################
trueVisibleGeneration_main = branches['RecoPFPTrueVisibleGeneration']
trueTrackID_main = branches['RecoPFPTrueTrackID']
trueVisibleParentTrackID_main = branches['RecoPFPTrueVisibleParentTrackID']

#############################
# How many entries are we working with?
#############################
nEntries = run.shape[0]
print('We are working with:', nEntries, 'entries')

#############################
# Rebuilt Hierarchy
#############################
newParentTrackID_main = []
newGen_main = []

We are working with: 98400 entries


In [5]:
#############################
# Find Primaries - Let's create our graphs!
#############################

modeDict = {
    "ADD_NEUTRINO"                 : True,
    "CHEAT_DIRECTION"              : True,
    "CHEAT_PID"                    : True, 
    "MAKE_PARTICLE_PARTICLE_LINKS" : True,
    "EDGE_FRACTION"                : 0.8,
    "DO_NORMALISATION"             : True,
    "IS_PRIMARY_TRAINING"          : False,
    "IS_HIGHER_TIER_TRAINING"      : False,
    "MAX_NODE_CLASS"               : 4
}

for iEvent in range(5) : 
            
    variables, y, parentPFPIndices, childPFPIndices = HigherTierFileHelper.readEvent(inputFileName, iEvent)
        
    eventDict = {
        "recoNuVertexX"                            : recoNuVertexX[iEvent], \
        "recoNuVertexY"                            : recoNuVertexY[iEvent], \
        "recoNuVertexZ"                            : recoNuVertexZ[iEvent], \
        "nParticles"                               : np.array(pfpTruePDG_main[iEvent]).shape[0], \
        "trackShowerScore"                         : np.array(trackShowerScore_main[iEvent], dtype='f'), \
        "nHits"                                    : np.array(nHits_main[iEvent], dtype='f'), \
        "charge"                                   : np.array(charge_main[iEvent], dtype='f'), \
        "vertexX"                                  : np.array(vertexX_main[iEvent], dtype='f'), \
        "vertexY"                                  : np.array(vertexY_main[iEvent], dtype='f'), \
        "vertexZ"                                  : np.array(vertexZ_main[iEvent], dtype='f'), \
        "trackEndX"                                : np.array(trackEndX_main[iEvent], dtype='f'), \
        "trackEndY"                                : np.array(trackEndY_main[iEvent], dtype='f'), \
        "trackEndZ"                                : np.array(trackEndZ_main[iEvent], dtype='f'), \
        "showerDirX"                               : np.array(showerDirX_main[iEvent], dtype='f'), \
        "showerDirY"                               : np.array(showerDirY_main[iEvent], dtype='f'), \
        "showerDirZ"                               : np.array(showerDirZ_main[iEvent], dtype='f'), \
        "ivysaurusMuon"                            : np.array(ivysaurusMuon_main[iEvent], dtype='f'), \
        "ivysaurusProton"                          : np.array(ivysaurusProton_main[iEvent], dtype='f'), \
        "ivysaurusPion"                            : np.array(ivysaurusPion_main[iEvent], dtype='f'), \
        "ivysaurusElectron"                        : np.array(ivysaurusElectron_main[iEvent], dtype='f'), \
        "ivysaurusPhoton"                          : np.array(ivysaurusPhoton_main[iEvent], dtype='f'), \
        "trackLength"                              : np.array(trackLength_main[iEvent], dtype='f'), \
        "displacement"                             : np.array(displacement_main[iEvent], dtype='f'), \
        "dca"                                      : np.array(dca_main[iEvent], dtype='f'), \
        "isNeutrinoPDG"                            : np.zeros(np.array(pfpTruePDG_main[iEvent]).shape), \
        "nuVertexEnergyAsymmetry"                  : np.array(nuVertexEnergyAsymmetry_main[iEvent], dtype='f'), \
        "nuVertexEnergyWeightedMeanRadialDistance" : np.array(nuVertexEnergyWeightedMeanRadialDistance_main[iEvent], dtype='f'), \
        "trueTrackID"                              : np.array(trueTrackID_main[iEvent]), \
        "trueVisibleParentTrackID"                 : np.array(trueVisibleParentTrackID_main[iEvent]), \
        "trueMomX"                                 : np.array(pfpTrueMomX_main[iEvent]), \
        "trueMomY"                                 : np.array(pfpTrueMomY_main[iEvent]), \
        "trueMomZ"                                 : np.array(pfpTrueMomZ_main[iEvent]), \
        "truePDG"                                  : np.array(pfpTruePDG_main[iEvent]), \
        "trueVisibleGeneration"                    : np.array(trueVisibleGeneration_main[iEvent])
    }

    data_pos, data_neg, data_FC, pfp_index = GraphBuilder.GraphBuilder(eventDict, modeDict)
    # If these are empty, we're going to have to do something about that...
    
    ##################################################
    # Calculate Scores of neutrino -> particle edges
    ##################################################
    if (data_FC.num_nodes > 1) :
        pred = gnn_model(data_FC)

        # Assuming that the neutrino is the last node added (which it is in this config)
        nuEdgeMask = (data_FC.edge_index[0] == (data_FC.num_nodes - 1))
        target_index = np.reshape(data_FC.edge_index[0][nuEdgeMask].detach().numpy(), -1)
        source_index = np.reshape(data_FC.edge_index[1][nuEdgeMask].detach().numpy(), -1)
        edge_index = torch.tensor([target_index, source_index], dtype=torch.long)

        edge_pred = primary_edge_classifier_model(pred, edge_index)

    ############################
    # Find Primaries
    ############################
    newParentTrackID = [-1] * eventDict["nParticles"]
    newParentTrackID[-1] = -999 # Nu has no parent
    newParentSelf = [-1] * eventDict["nParticles"]
    newGen = [-1] * eventDict["nParticles"]
    newParentSelf[-1] = -999 # Nu has no parent
    newGen[-1] = 1 

    particle_tiers = [[]]

    for iParticle in range(eventDict["nParticles"]) :
    
        # Move on if it is the neutrino
        if (eventDict['isNeutrinoPDG'][iParticle] == 1) :
            continue
            
        #print('iParticle:', iParticle)
                        
        # Node is not in graph (inf ivy or 2D?)
        if (len(np.where(pfp_index == iParticle)[0]) == 0) :
            continue 
    
        primary_GNN_score = edge_pred[np.where(pfp_index == iParticle)[0][0]].item()
                
        #print('primary_GNN_score:', primary_GNN_score)
    
        if (primary_GNN_score > PRIMARY_THRESHOLD) :
            newParentTrackID[iParticle] = 0
            newParentSelf[iParticle] = eventDict["nParticles"] # nu has self == nParticles
            newGen[iParticle] = 2
            particle_tiers[0].append(iParticle)

    ############################
    # Find Higher Tiers
    ############################    
    while (len(particle_tiers[-1]) != 0) :
        
        tier = []
        
        for iChild in range(eventDict["nParticles"]) :
            
            if (eventDict["isNeutrinoPDG"][iChild] == 1) :
                continue
            
            # Has it been assigned yet?
            if (newGen[iChild] != -1) :
                continue
                
            highestOtherScore = -1
            iFoundParent = -1
        
            for iParent in particle_tiers[-1]:
                
                # If we've previously been unable to find any primaries
                if (eventDict["isNeutrinoPDG"][iParent] == 1) :
                    continue
                
                linkIndex = HigherTierFileHelper.getLinkIndex(parentPFPIndices, childPFPIndices, iParent, iChild) 
                
                # If the particle-particle link hasn't been saved in the analyser
                if (linkIndex < 0) :
                    continue
                    
                linkVariables = variables[linkIndex].reshape(-1,12)
                linkTruth = y[linkIndex]
                                
                y_pred = other_edge_classifier_model.predict(linkVariables, verbose=2)
                
                if ((y_pred > highestOtherScore) and (y_pred > OTHER_THRESHOLD)) :
                    highestOtherScore = y_pred
                    iFoundParent = iParent
                
            if (iFoundParent != -1) :
                newParentTrackID[iChild] = trueTrackID_main[iEvent][iFoundParent]
                newParentSelf[iChild] = recoSelf_main[iEvent][iFoundParent]
                newGen[iChild] = len(particle_tiers) + 2
                tier.append(iChild)
                    
        particle_tiers.append(tier)
        
    ######################################
    # Say that any remaining are primaries
    ###################################### 
#     for iParticle in range(eventDict["nParticles"]) :
    
#         # Move on if it is the neutrino
#         if (eventDict['isNeutrinoPDG'][iParticle] == 1) :
#             continue
            
#         if (newGen[iParticle] == -1) :
#             newParentTrackID[iParticle] = 0
#             newParentSelf[iParticle] = eventDict["nParticles"] # nu has self == nParticles
#             newGen[iParticle] = 2
#             particle_tiers[0].append(iParticle)    
        
#     print('------------------------')
#     print('newParentTrackID:     ', newParentTrackID)
#     print('newGen:         ', newGen)
#     print('trueVisibleGeneration:', trueVisibleGeneration_main[iEvent])
#     print('particle_tiers:', particle_tiers)
#     print('------------------------')
                
    newParentTrackID_main.append(newParentTrackID)
    newGen_main.append(newGen)    

1/1 - 0s - 32ms/epoch - 32ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step


/var/folders/1g/qpfx8_ps0dn6405cqjp61k5r0000gn/T/ipykernel_38376/2607316742.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:277.)
  edge_index = torch.tensor([target_index, source_index], dtype=torch.long)


1/1 - 0s - 6ms/epoch - 6ms/step
1/1 - 0s - 6ms/epoch - 6ms/step


In [6]:
#############################
# Metrics!
#############################

n_true_primary = 0
n_true_secondary = 0
n_true_higher = 0

# Old!
n_correct_parent_primary_old = 0
n_tagged_as_primary_primary_old = 0
n_incorrect_parent_primary_old = 0

n_correct_parent_secondary_old = 0
n_tagged_as_primary_secondary_old = 0
n_incorrect_parent_secondary_old = 0

n_correct_parent_higher_old = 0
n_tagged_as_primary_higher_old = 0
n_incorrect_parent_higher_old = 0

# NEW!
n_correct_parent_primary_new = 0
n_tagged_as_primary_primary_new = 0
n_incorrect_parent_primary_new = 0
n_not_tagged_primary_new = 0 

n_correct_parent_secondary_new = 0
n_tagged_as_primary_secondary_new = 0
n_incorrect_parent_secondary_new = 0
n_not_tagged_secondary_new = 0 

n_correct_parent_higher_new = 0
n_tagged_as_primary_higher_new = 0
n_incorrect_parent_higher_new = 0
n_not_tagged_higher_new = 0 

for iEvent in range(5) : 
    
    # Only look at 3D particles! 
    vertexX_np = np.array(vertexX_main[iEvent])
    threeDMask = (vertexX_np > -900)
    
    # Truth
    trueVisibleGeneration_np = np.array(trueVisibleGeneration_main[iEvent])[threeDMask]
    trueVisibleParentTrackID_np = np.array(trueVisibleParentTrackID_main[iEvent])[threeDMask]
    # Old 
    recoGeneration_np = np.array(recoGeneration_main[iEvent])[threeDMask]
    # New
    newParentTrackID_np = np.array(newParentTrackID_main[iEvent][:-1])[threeDMask]
    newGen_np = np.array(newGen_main[iEvent][:-1])[threeDMask]
    
#     print('////////////////////////////////////////////')
#     print('//////////////// NEW EVENT ////////////////')
#     print('////////////////////////////////////////////')    
#     print('')
#     print('Truth')
#     print('---------')
#     print('trueVisibleParentTrackID_np:', trueVisibleParentTrackID_np)
#     print('trueVisibleGeneration_np:', trueVisibleGeneration_np)
#     print('')
#     print('Old')
#     print('---------')
#     print('recoGeneration_np:', recoGeneration_np)
#     print('')
#     print('New')
#     print('---------')
#     print('newParentTrackID_np:', newParentTrackID_np)
#     print('newGen_main:', newGen_np)

    #########################
    # Get tier masks
    #########################
    true_primary_mask = (trueVisibleGeneration_np == 2)
    true_secondary_mask = (trueVisibleGeneration_np == 3)
    true_higher_mask = np.logical_not(np.logical_or(true_primary_mask, true_secondary_mask))

    #############################################
    # Get metrics for this event - debugging
    #############################################
    # Totals
    this_true_primary = np.count_nonzero(true_primary_mask)
    this_true_secondary = np.count_nonzero(true_secondary_mask)
    this_true_higher = np.count_nonzero(true_higher_mask)
    # Old
    this_correct_parent_primary_old = np.count_nonzero(recoGeneration_np[true_primary_mask] == 2)
    this_tagged_as_primary_primary_old = 0
    this_incorrect_parent_primary_old = np.count_nonzero(recoGeneration_np[true_primary_mask] != 2)
    this_correct_parent_secondary_old = -1
    this_tagged_as_primary_secondary_old = np.count_nonzero(recoGeneration_np[true_secondary_mask] == 2)
    this_incorrect_parent_secondary_old = -1
    this_correct_parent_higher_old = -1
    this_tagged_as_primary_higher_old = np.count_nonzero(recoGeneration_np[true_higher_mask] == 2)
    this_incorrect_parent_higher_old = -1
    # New
    this_correct_parent_primary_new = np.count_nonzero(newGen_np[true_primary_mask] == 2)
    this_tagged_as_primary_primary_new = 0
    this_not_tagged_primary_new = np.count_nonzero(newGen_np[true_primary_mask] == -1)
    this_incorrect_parent_primary_new = np.count_nonzero(np.logical_and(newGen_np[true_primary_mask] != 2, \
                                                                        newGen_np[true_primary_mask] != -1)) 
    this_correct_parent_secondary_new = np.count_nonzero(newParentTrackID_np[true_secondary_mask] == trueVisibleParentTrackID_np[true_secondary_mask])
    this_tagged_as_primary_secondary_new = np.count_nonzero(newGen_np[true_secondary_mask] == 2)
    this_not_tagged_secondary_new = np.count_nonzero(newGen_np[true_secondary_mask] == -1)
    this_incorrect_parent_secondary_new = np.count_nonzero(np.logical_not(np.logical_or(newParentTrackID_np[true_secondary_mask] == trueVisibleParentTrackID_np[true_secondary_mask], \
                                                                                        np.logical_or(newGen_np[true_secondary_mask] == 2, \
                                                                                                      newGen_np[true_secondary_mask] == -1))))
    this_correct_parent_higher_new = np.count_nonzero(newParentTrackID_np[true_higher_mask] == trueVisibleParentTrackID_np[true_higher_mask])
    this_tagged_as_primary_higher_new = np.count_nonzero(newGen_np[true_higher_mask] == 2)
    this_not_tagged_higher_new = np.count_nonzero(newGen_np[true_higher_mask] == -1)
    this_incorrect_parent_higher_new = np.count_nonzero(np.logical_not(np.logical_or(newParentTrackID_np[true_higher_mask] == trueVisibleParentTrackID_np[true_higher_mask], \
                                                                                     np.logical_or(newGen_np[true_higher_mask] == 2, \
                                                                                                   newGen_np[true_higher_mask] == -1))))
    
    #############################################
    # Print metrics for this event - debugging
    #############################################   
#     print('------------------------------------------------')
#     print('OLD - True Gen  | Primary | Secondary | Higher |')
#     print('------------------------------------------------')
#     print('Correct parent  |' + str(this_correct_parent_primary_old) + '|' + str(this_correct_parent_secondary_old) + '|' + str(this_correct_parent_higher_new) + '|')
#     print('False primary   |' + str(this_tagged_as_primary_primary_old) + '|' + str(this_tagged_as_primary_secondary_old) + '|' + str(this_tagged_as_primary_higher_old) + '|')
#     print('Incorrect parent|' + str(this_incorrect_parent_primary_old) + '|' + str(this_incorrect_parent_secondary_old) + '|' + str(this_incorrect_parent_higher_old) + '|')
#     print('Total           |' + str(this_true_primary) + '|' + str(this_true_secondary) + '|' + str(this_true_higher) + '|')
#     print('------------------------------------------------')
#     print('')
#     print('------------------------------------------------')
#     print('NEW - True Gen  | Primary | Secondary | Higher |')
#     print('------------------------------------------------')
#     print('Correct parent  |' + str(this_correct_parent_primary_new) + '|' + str(this_correct_parent_secondary_new) + '|' + str(this_correct_parent_higher_new) + '|')
#     print('False primary   |' + str(this_tagged_as_primary_primary_new) + '|' + str(this_tagged_as_primary_secondary_new) + '|' + str(this_tagged_as_primary_higher_new) + '|')
#     print('Incorrect parent|' + str(this_incorrect_parent_primary_new) + '|' + str(this_incorrect_parent_secondary_new) + '|' + str(this_incorrect_parent_higher_new) + '|')
#     print('Total           |' + str(this_true_primary) + '|' + str(this_true_secondary) + '|' + str(this_true_higher) + '|')
#     print('------------------------------------------------')
    
    #############################################
    # Add metrics to global
    #############################################
    n_true_primary += this_true_primary
    n_true_secondary += this_true_secondary
    n_true_higher += this_true_higher
    n_correct_parent_primary_old += this_correct_parent_primary_old
    n_tagged_as_primary_primary_old += this_tagged_as_primary_primary_old
    n_incorrect_parent_primary_old += this_incorrect_parent_primary_old
    n_correct_parent_secondary_old += this_correct_parent_secondary_old
    n_tagged_as_primary_secondary_old += this_tagged_as_primary_secondary_old
    n_incorrect_parent_secondary_old += this_incorrect_parent_secondary_old
    n_correct_parent_higher_old += this_correct_parent_higher_old
    n_tagged_as_primary_higher_old += this_tagged_as_primary_higher_old
    n_incorrect_parent_higher_old += this_incorrect_parent_higher_old
    n_correct_parent_primary_new += this_correct_parent_primary_new
    n_tagged_as_primary_primary_new += this_tagged_as_primary_primary_new
    n_incorrect_parent_primary_new += this_incorrect_parent_primary_new
    n_not_tagged_primary_new += this_not_tagged_primary_new
    n_correct_parent_secondary_new += this_correct_parent_secondary_new
    n_tagged_as_primary_secondary_new += this_tagged_as_primary_secondary_new
    n_incorrect_parent_secondary_new += this_incorrect_parent_secondary_new
    n_not_tagged_secondary_new += this_not_tagged_secondary_new
    n_correct_parent_higher_new += this_correct_parent_higher_new
    n_tagged_as_primary_higher_new += this_tagged_as_primary_higher_new
    n_incorrect_parent_higher_new += this_incorrect_parent_higher_new
    n_not_tagged_higher_new += this_not_tagged_higher_new
    
    
    
#############################################
# Print metrics for this event - debugging
#############################################   
print('')
print('-------------------------------------------------')
print('OLD - True Gen   | Primary | Secondary | Higher |')
print('-------------------------------------------------')
print('Correct parent   |' + str(n_correct_parent_primary_old) + str(' '* (9 - len(str(n_correct_parent_primary_old)))) + \
                       '|' + str(n_correct_parent_secondary_old) + str(' '* (11 - len(str(n_correct_parent_secondary_old)))) + \
                       '|' + str(n_correct_parent_higher_new) + str(' '* (8 - len(str(n_correct_parent_higher_new)))) + \
                       '|')
print('False primary    |' + str(n_tagged_as_primary_primary_old) + str(' '* (9 - len(str(n_tagged_as_primary_primary_old)))) + \
                       '|' + str(n_tagged_as_primary_secondary_old) + str(' '* (11 - len(str(n_tagged_as_primary_secondary_old)))) + \
                       '|' + str(n_tagged_as_primary_higher_old) + str(' '* (8 - len(str(n_tagged_as_primary_higher_old)))) + \
                       '|')
print('Incorrect parent |' + str(n_incorrect_parent_primary_old) +str(' '* (9 - len(str(n_incorrect_parent_primary_old)))) + \
                       '|' + str(n_incorrect_parent_secondary_old) + str(' '* (11 - len(str(n_incorrect_parent_secondary_old)))) + \
                       '|' + str(n_incorrect_parent_higher_old) +str(' '* (8 - len(str(n_incorrect_parent_higher_old)))) + \
                       '|')
print('------------------------------------------------')
print('Total            |' + str(n_true_primary) + str(' '* (9 - len(str(n_true_primary)))) + \
                       '|' + str(n_true_secondary) + str(' '* (11 - len(str(n_true_secondary)))) + \
                       '|' + str(n_true_higher) + str(' '* (8 - len(str(n_true_higher)))) + \
                       '|')
print('------------------------------------------------')
print('')
print('-------------------------------------------------')
print('NEW - True Gen   | Primary | Secondary | Higher |')
print('-------------------------------------------------')
print('Correct parent   |' + str(n_correct_parent_primary_new) + str(' '* (9 - len(str(n_correct_parent_primary_new)))) + \
                       '|' + str(n_correct_parent_secondary_new) + str(' '* (11 - len(str(n_correct_parent_secondary_new)))) + \
                       '|' + str(n_correct_parent_higher_new) + str(' '* (8 - len(str(n_correct_parent_higher_new)))) + \
                       '|')
print('False primary    |' + str(n_tagged_as_primary_primary_new) + str(' '* (9 - len(str(n_tagged_as_primary_primary_new)))) + \
                       '|' + str(n_tagged_as_primary_secondary_new) + str(' '* (11 - len(str(n_tagged_as_primary_secondary_new)))) + \
                       '|' + str(n_tagged_as_primary_higher_new) + str(' '* (8 - len(str(n_tagged_as_primary_higher_new)))) + \
                       '|')
print('Incorrect parent |' + str(n_incorrect_parent_primary_new) + str(' '* (9 - len(str(n_incorrect_parent_primary_new)))) + \
                       '|' + str(n_incorrect_parent_secondary_new) + str(' '* (11 - len(str(n_incorrect_parent_secondary_new)))) + \
                       '|' + str(n_incorrect_parent_higher_new) + str(' '* (8 - len(str(n_incorrect_parent_higher_new)))) + \
                       '|')
print('Not tagged       |' + str(n_not_tagged_primary_new) + str(' '* (9 - len(str(n_not_tagged_primary_new)))) + \
                       '|' + str(n_not_tagged_secondary_new) + str(' '* (11 - len(str(n_not_tagged_secondary_new)))) + \
                       '|' + str(n_not_tagged_higher_new) + str(' '* (8 - len(str(n_not_tagged_higher_new)))) + \
                       '|')
print('-------------------------------------------------')
print('Total            |' + str(n_true_primary) + str(' '* (9 - len(str(n_true_primary)))) + \
                       '|' + str(n_true_secondary) + str(' '* (11 - len(str(n_true_secondary)))) + \
                       '|' + str(n_true_higher) + str(' '* (8 - len(str(n_true_higher)))) + \
                       '|')
print('-------------------------------------------------')


-------------------------------------------------
OLD - True Gen   | Primary | Secondary | Higher |
-------------------------------------------------
Correct parent   |14       |-5         |0       |
False primary    |0        |1          |0       |
Incorrect parent |0        |-5         |-5      |
------------------------------------------------
Total            |14       |2          |0       |
------------------------------------------------

-------------------------------------------------
NEW - True Gen   | Primary | Secondary | Higher |
-------------------------------------------------
Correct parent   |13       |1          |0       |
False primary    |0        |0          |0       |
Incorrect parent |0        |0          |0       |
Not tagged       |1        |1          |0       |
-------------------------------------------------
Total            |14       |2          |0       |
-------------------------------------------------


In [7]:
print(str(len('jam')) + str(' '*2) + 'hi')

3  hi
